In [57]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [58]:
# zapobieganie wypisywania notacji naukowej przez numpy
np.set_printoptions(suppress=True)

https://stat.gov.pl/obszary-tematyczne/ceny-handel/wskazniki-cen/wskazniki-cen-towarow-i-uslug-konsumpcyjnych-pot-inflacja-/miesieczne-wskazniki-cen-towarow-i-uslug-konsumpcyjnych-od-1982-roku/

### Załadowanie pliku

In [2]:
### Załadowanie dysku
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
data_file_path = "/content/drive/My Drive/Colab Notebooks/miesieczne_wskazniki_cen_towarow_i_uslug_konsumpcyjnych_od_1982_roku.xlsx"
try:
  df = pd.read_excel(data_file_path)
  if "Sposób prezentacji" in df.columns:
      # Filtrowanie danych
      df_poprzedni_miesiac_100 = df[df["Sposób prezentacji"] == "Poprzedni miesiąc = 100"].copy()

      print("Wczytano dane i przefiltrowano według 'Sposób prezentacji' = 'Poprzedni miesiąc=100'.")
  else:
      print("Kolumna 'Sposób prezentacji' nie została znaleziona w pliku Excel.")
except FileNotFoundError:
    print(f"Błąd: Plik nie znaleziony pod ścieżką {data_file_path}")
except Exception as e:
    print(f"Wystąpił błąd podczas wczytywania pliku: {e}")

Wczytano dane i przefiltrowano według 'Sposób prezentacji' = 'Poprzedni miesiąc=100'.


### Filtracja danych

In [60]:
# TODO: sprawdz co oznacza Flaga 'a' w roku 2000 i 2001
#df_poprzedni_miesiac_100[["Sposób prezentacji", "Rok", "Miesiąc", "Wartość", "Flaga"]]
data = df_poprzedni_miesiac_100[["Rok", "Miesiąc", "Wartość"]]
# pozbywamy się wierszy z wartościami NaN, występujące w roku 2025
data = data.dropna(subset=["Wartość"])
data = data.sort_values(by=['Rok','Miesiąc'], ascending=[1,1])

data_array = data.to_numpy()
# print(data.head(20))

Mając różnice pomiędzy następnymi miesiącami, usuwamy oulinery, które pokazują dużą różnicę inflacji.

In [61]:
# średnia arytmetyczna
mean = np.mean(data_array[:, 2:3], axis=0)
# odchylenie standardowe
sd = np.std(data_array[:, 2:3], axis=0)
top_limit = mean + 3 * sd
bottom_limit = mean - 3 * sd

# wypiszmy dane, które zostaną usunięte (oulinery)
for row in data_array:
  if (row[2] <= bottom_limit) or (row[2] >= top_limit):
    print(row)

data_array_filtered = [row for row in data_array if (row[2] > bottom_limit)]
data_array_filtered = [row for row in data_array_filtered if (row[2] < top_limit)]

[1982.     2.   136.3]
[1989.     8.   139.5]
[1989.     9.   134.4]
[1989.    10.   154.8]
[1989.    11.   122.4]
[1990.     1.   179.6]
[1990.     2.   123.8]


### Konwersja danych

inclacja - gdy "Wartość" > 100

deflacja - gdy "Wartość" < 100

In [62]:
# kolumna 0, rok
# kolumna 1, miesiąc
# kolumna 2, wartość względem grudnia 1981
data_len = len(data)
data_absolute_inflation = np.zeros((len(data), 3))
absolute_result = 1.0
idx = 0
for row in data_array:
  absolute_result = absolute_result + absolute_result * ((float(row[2])-100.0) / 100.0)
  data_absolute_inflation[idx] = [int(row[0]), int(row[1]), absolute_result]
  idx += 1
print(data_absolute_inflation)

[[1982.            1.            1.066     ]
 [1982.            2.            1.452958  ]
 [1982.            3.            1.53722956]
 ...
 [2025.            2.         3590.96204085]
 [2025.            3.         3598.14396493]
 [2025.            4.         3612.53654079]]


Wydaje mi się, że lepiej będzie można wykorzystać dane data_absolute_date_inflation, ponieważ tam mamy jedną kolumnę określającą czas, ale nie jestem pewien.

In [63]:
# kolumna 0, ilość miesięcy od grudnia 1982 (1 = dla styczeń 1982, 2 dla luty 1982, ..., 13 dla styczeń 1983, itd.)
# kolumna 1, wartość względem grudnia 1981
data_len = len(data)
data_absolute_date_inflation = np.zeros((len(data), 2))
absolute_result = 1.0
idx = 0
for row in data_array:
  absolute_result = absolute_result + absolute_result * ((float(row[2])-100.0) / 100.0)
  data_absolute_date_inflation[idx] = [(int(row[0])-1982)*12 + int(row[1]), absolute_result]
  idx += 1
print(data_absolute_date_inflation)

[[   1.            1.066     ]
 [   2.            1.452958  ]
 [   3.            1.53722956]
 ...
 [ 518.         3590.96204085]
 [ 519.         3598.14396493]
 [ 520.         3612.53654079]]


### Podział danych na uczące i testujące

In [64]:
## losowy podział danych
# X = data_absolute_inflation[:, 0:2] # daty
# y = data_absolute_inflation[:, 2]   # inflacja
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Podział danych na 80% danych uczących, tj. od styczeń 1882 do września 2016 roku.

Nie ma sensu chyba brać innego podziału danych,

In [86]:
procent_danych_testowych = 0.8

X = data_absolute_inflation[:, 0:2] # daty
y = data_absolute_inflation[:, 2]   # inflacja
x_len = int(len(X) * procent_danych_testowych)
y_len = int(len(y) * procent_danych_testowych)
X_train = X[:x_len]
X_test = X[x_len:]
y_train = y[:y_len]
y_test = y[y_len:]
print(f"Przedział dat danych trenujących: 0{int(X_train[0][1])}.{int(X_train[0][0])} - 0{int(X_train[-1][1])}.{int(X_train[-1][0])}")
print(f"Przedział dat danych testowych:   0{int(X_test[0][1])}.{int(X_test[0][0])} - 0{int(X_test[-1][1])}.{int(X_test[-1][0])}")

Przedział dat danych trenujących: 01.1982 - 08.2016
Przedział dat danych testowych:   09.2016 - 04.2025
